# Один из возможных алгоритмов скрейпинга

* указать в коде адрес интересующего сайта: откуда вы хотите скачать данные?
для этого используется библиотека requests

* сохранить веб-страницу (html-код страницы)

* выбрать данные, которые нужно собрать (используется BeautifulSoup)

* записать данные в csv-файл.

Если нужно соскрейпить несколько страниц - повторяем процесс для каждой из них.

**Наша задача:** выгрузить недавние новости в датафрейм `pandas`, чтобы потом сохранить все в csv-файл.

Сначала сгрузим весь html-код страницы и сохраним его в отдельную переменную. Для этого нам понадобится библиотека `requests`. Документация: https://requests.readthedocs.io/en/latest/

Импортируем её:

In [ ]:
import requests

Сохраним ссылку на главную страницу сайта в переменную `url` для удобства и выгрузим страницу. (Разумеется, это будет работать при подключении к интернету. Если соединение будет отключено, Python выдаст `NewConnectionError`).

In [ ]:
url = 'https://nplus1.ru/' # сохраняем
page = requests.get(url) # загружаем страницу по ссылке

Если мы просто посмотрим на объект, мы ничего особенного не увидим:

In [ ]:
page  # response 200 - страница загружена

**BeautifulSoup** - python-библиотека для синтаксического разбора файлов HTML/XML. В веб-разработке «суп из тегов» (tag soup) - это слово для синтаксически или структурно некорректного HTML, написанного для веб-страницы.

Документация: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

Импортируем функцию `BeautifulSoup` из библиотеки `bs4` (от *beautifulsoup4*) и заберём со страницы `page` код html в виде текста. 

Сохраним в переменную 'soup' весь HTML-код страницы. HTML-код - это "дерево тегов", формирующее контент страницы.

In [ ]:
from bs4 import BeautifulSoup 
# название - отсылка к песне про суп из Алисы в стране чудес https://aliceinwonderland.fandom.com/wiki/Turtle_Soup

In [ ]:
soup = BeautifulSoup(page.text, 'html')

Если выведем `soup` на экран, мы увидим то же самое, что в режиме разработчика или в режиме происмотра исходного кода (`view-source` через *Ctrl+U* в Google Chrome).

In [ ]:
soup

Для просмотра выглядит не очень удобно.  «Причешем» наш `soup` – воспользуемся методом `.prettify()` в сочетании с функцией `print()`.

In [ ]:
print(soup.prettify())

В такой выдаче ориентироваться гораздо удобнее (но при желании, то же можно увидеть в браузере, на большом экране).

Чтобы сгрузить все новости с главной страницы сайта, нужно собрать все ссылки на страницы с этими новостями. Ссылки в html-файле всегда заключены в тэг `<a></a>` и имеют атрибут `href`. 

**Функция `soup.find('a')`** найдет первый в дереве тег \<a>.
Если нам нужно найти не только первый элемент, а все элементы по определенному признаку, следует использовать функцию **`soup.find_all('a')`**

In [ ]:
for link in soup.find_all('a'):
    print(link.get('href'))

Ссылок много. Но нам нужны только новости – ссылки, которые начинаются со слова `/news`. Добавим условие: будем выбирать только те ссылки, в которых есть `/news`. Создадим пустой список `urls` и будем добавлять в него только ссылки, которые удовлетворяют этому условию.

In [ ]:
urls = []

for link in soup.find_all('a'):
#     print(link)
    if '/news' in link.get('href'):
        if 'https://' in link.get('href'):
            urls.append(link.get('href'))
urls

Теперь наша задача сводится к следующему: изучить одну страницу с новостью, научиться из нее вытаскивать текст и всю необходимую информацию, а потом применить весь набор действий к каждой ссылке из `full_urls` в цикле. Посмотрим на новость с индексом 1, у вас может быть другая, новости обновляются.

In [ ]:
url0 = urls[1]

page0 = requests.get(url0)
soup0 = BeautifulSoup(page0.text, 'html')
soup0

В коде каждой страницы с новостью есть часть с мета-информацией: датой, именем автора и проч. Такая информация окружена тэгом `<meta></meta>`. Посмотрим:

In [ ]:
soup0.find_all('meta')

Из этого списка нам нужны части с именем автора, датой, заголовком и кратким описанием. Воспользуемся поиском по атрибуту `name`. Передадим функции `find_all()` в качестве аргумента словарь с названием и значением атрибута: 

In [ ]:
soup0.find_all('meta', {'name' : 'author'}) # например, автор

Теперь выберем единственный элемент полученного списка (с индексом 0):

In [ ]:
soup0.find_all('meta', {'name' : 'author'})[0]

Нам нужно вытащить из этого объекта `content` – имя автора. Посмотрим на атрибуты:

In [ ]:
soup0.find_all('meta', {'name' : 'author'})[0].attrs

Как получить отсюда `content`? Очень просто, ведь это словарь! А доставать из словаря значение по ключу мы умеем.

In [ ]:
author = soup0.find_all('meta', {'name' : 'author'})[0].get('content')
author

Аналогичным образом извлечем дату, заголовок и описание.

In [ ]:
date = soup0.find_all('meta', {'itemprop' : 'datePublished'})[0].get('content')
title = soup0.find_all('meta', {'property' : 'og:title'})[0].get('content')

Теперь осталось совсем чуть-чуть. Написать готовую функцию для всех проделанных нами действий и применить ее в цикле для всех ссылок в списке `full_urls`. Напишем! Аргументом функции будет ссылка на новость, а возвращать она будет текст новости и всю необходимую информацию (дата, автор, сложность и проч.). Скопируем все строки кода выше.

In [ ]:
def GetNews(url0):
    """
    Возвращает кортеж с url0, date, author, description, title, final_text, rubrics, diff.
    Параметры:
    
    url0 - ссылка на новость (строка).
    """
    page0 = requests.get(url0)
    soup0 = BeautifulSoup(page0.text, 'lxml')
    
    try:
        author = soup0.find_all('meta', {'name' : 'author'})[0].get('content')
    except:
        author = None
    date = soup0.find_all('meta', {'itemprop' : 'datePublished'})[0].get('content')
    title = soup0.find_all('meta', {'property' : 'og:title'})[0].get('content')
 
    
    return url0, date, author, title

Уфф. Осталось применить ее в цикле. Но давайте не будем спешить: импортируем функцию `sleep` для задержки, чтобы на каждой итерации цикла, прежде чем перейти к следующей новости, Python ждал несколько секунд. Во-первых, это нужно, чтобы сайт «не понял», чтобы мы его грабим, да еще автоматически. Во-вторых, с небольшой задержкой всегда есть гарантия, что страница прогрузится (сейчас это пока не очень важно, но особенно актуально будет, когда будем обсуждать встраивание в браузер с Selenium). Приступим.

In [ ]:
from time import sleep

In [ ]:
len(urls)

In [ ]:
news = [] # это будет список из кортежей, в которых будут храниться данные по каждой новости

for link in urls[:50]:
    print(link)
    res = GetNews(link)
    news.append(res)
        
    sleep(1) # задержка в 3 секунды

In [ ]:
news[0]

Так теперь выглядит первый элемент списка:

Импортируем `pandas` и создадим датафрейм из списка кортежей: 

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(news)

In [ ]:
df.head(10)

Переименуем столбцы в базе.

In [ ]:
df.columns = ['link', 'date', 'author', 'title']

In [ ]:
df.head(2)

Всё! Сохраняем датафрейм в файл. Для разнообразия сохраним в Excel:

In [ ]:
df.to_excel('nplus-news.xlsx', index=False)